<a href="https://colab.research.google.com/github/4L3M4R/cerbero/blob/main/cerbero-pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================================
#             CERBERO PREMARKET
#   Descarga noticias y calcula sentiment
#   (Actualiza el día anterior en los archivos *_datos.txt)
# ===============================================

import os
import pandas as pd
import feedparser
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from datetime import datetime, timedelta
from urllib.parse import quote

# ===============================================
#             CONFIGURACIÓN
# ===============================================

# Lista de activos
activos = {}
with open("activos.txt", "r") as f:
    for line in f:
        symbol, source, search_name = line.strip().split(":")
        activos[symbol.strip()] = {
            "source": source.strip().lower(),
            "search_name": search_name.strip()
        }

# Inicialización de analizadores de sentimiento
nltk.download('vader_lexicon')
vader_analyzer = SentimentIntensityAnalyzer()
finbert_tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
finbert_model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

# ===============================================
#           FUNCIONES AUXILIARES
# ===============================================

def registrar_log(message, log_file="run_summary_pre.log"):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_file, "a") as log:
        log.write(f"[{timestamp}] {message}\n")

def descargar_noticias_y_calcular_sentiment(symbol, search_name):
    os.makedirs("logs", exist_ok=True)

    query = quote(search_name)
    feed = feedparser.parse(f"https://news.google.com/rss/search?q={query}")
    noticias = []
    for entry in feed.entries:
        noticias.append({
            "timestamp": entry.published,
            "title": entry.title,
            "link": entry.link
        })

    df = pd.DataFrame(noticias)
    if df.empty:
        registrar_log(f"{symbol} - No se encontraron noticias")
        return

    # Calcular sentimiento con VADER (y placeholder para FinBERT)
    df["vader_sentiment"] = df["title"].apply(lambda x: vader_analyzer.polarity_scores(x)["compound"])
    df["finbert_sentiment"] = 0  # Puedes reemplazar con análisis FinBERT si lo deseas

    # Calcular promedios
    vader_promedio = df["vader_sentiment"].mean()
    finbert_promedio = df["finbert_sentiment"].mean()

    # Actualizar el archivo de datos
    actualizar_sentimiento(symbol, vader_promedio, finbert_promedio)

    # Guardar noticias procesadas
    today_str = pd.Timestamp.utcnow().date()
    df.to_csv(f"logs/{symbol}_nuevas_agregadas_{today_str}.csv", index=False)
    registrar_log(f"{symbol} - Guardadas {len(df)} noticias con sentiment")

def actualizar_sentimiento(symbol, vader_promedio, finbert_promedio):
    """Actualiza la línea del DÍA ANTERIOR en el archivo datos_{symbol}.txt con el sentimiento promedio."""
    import pandas as pd
    import os
    file_path = f"{symbol}_datos.txt"
    if not os.path.exists(file_path):
        print(f"⚠️ Archivo no encontrado: {file_path}")
        registrar_log(f"{symbol} - Archivo de datos no encontrado: {file_path}")
        return

    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        registrar_log(f"{symbol} - Error leyendo archivo {file_path}: {e}")
        print(f"❌ Error leyendo {file_path}: {e}")
        return

    print(f"✅ Archivo cargado correctamente: {file_path}")

    # --- Identificar columna de fecha ---
    date_col = None
    for col in df.columns:
        if col.lower() in ["timestamp", "date", "fecha", "day"]:
            date_col = col
            break

    if date_col is None:
        registrar_log(f"{symbol} - No se encontró columna de fecha en {file_path}")
        print("⚠️ No se encontró columna de fecha, no se actualiza.")
        return

    df["timestamp"] = pd.to_datetime(df[date_col], errors="coerce").dt.date

    # --- Buscar la línea del día ANTERIOR ---
    ayer = (datetime.now() - timedelta(days=1)).date()
    mask = df["timestamp"] == ayer

    if mask.any():
        print(f"🟢 Actualizando línea existente para {ayer} en {symbol}")
        df.loc[mask, "vader_sentiment"] = vader_promedio
        df.loc[mask, "finbert_sentiment"] = finbert_promedio
        registrar_log(f"{symbol} - Sentimiento actualizado para {ayer}: Vader={vader_promedio:.3f}, FinBERT={finbert_promedio:.3f}")
    else:
        print(f"🟡 No existe línea para {ayer}, agregando nueva fila.")
        nueva_fila = pd.DataFrame({
            "timestamp": [ayer],
            "vader_sentiment": [vader_promedio],
            "finbert_sentiment": [finbert_promedio],
        })
        df = pd.concat([df, nueva_fila], ignore_index=True)
        registrar_log(f"{symbol} - Nueva fila agregada para {ayer}")

    # --- Guardar el archivo actualizado ---
    df.to_csv(file_path, index=False)
    print(f"💾 Archivo actualizado y guardado: {file_path}")

# ===============================================
#           EJECUCIÓN PRINCIPAL
# ===============================================

for symbol, info in activos.items():
    descargar_noticias_y_calcular_sentiment(symbol, info["search_name"])

print("✅ Completed premarket (sentiment del día anterior actualizado)")
